In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile
from tqdm import tqdm

zf = zipfile.ZipFile('drive/MyDrive/DL_dataset/cats_dogs.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

100%|██████████| 25004/25004 [00:14<00:00, 1753.30it/s]


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import numpy as np

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from pytorchtools_st import EarlyStopping

In [13]:
cats_dogs = ImageFolder('cats_dogs/train', transform=transforms.Compose([transforms.Resize([224, 224]), 
                                                                               transforms.ToTensor(),
                                                                               transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]))
train_size = int(np.floor(len(cats_dogs)*0.8))
test_size = len(cats_dogs) - train_size 

dataset_train, test_val = random_split(cats_dogs, [train_size, test_size]) # Разделение на тренировчоную и вылидационные выборки

train_loader = DataLoader(dataset_train, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_val, batch_size = len(test_val), shuffle = True)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 10)

for param in model.parameters():
  param.requires_grad = False

for param in model.fc.parameters(): # Обучаются только параметры линейных слоев
  param.requires_grad = True
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

early_stop = EarlyStopping(delta = 0.5, patience = 3, verbose = False) # Ранняя остановка

n_epochs = 500

train_history_loss = list()
epochs_history = list()

for epoch in range(1, n_epochs + 1):
  epoch_loss = 0

  model.train() # явно указываем обучение
  for X_batch, y_batch in train_loader:
    X_batch, y_batch = X_batch.to('cpu'), y_batch.to('cpu')
    # forward
    out = model(X_batch)
    # loss
    batch_loss = criterion(out, y_batch)
    # backward
    batch_loss.backward()
    # optimization
    optimizer.step()
    optimizer.zero_grad()
    epoch_loss += batch_loss.item()

  train_history_loss.append(epoch_loss/len(train_loader))
  epochs_history.append(epoch)

  if epoch % 2 == 0:
    print(f'Epoch: [{epoch}/{n_epochs}], Train_loss: {train_history_loss[-1]}')

  early_stop(train_history_loss[-1], model)
  
  if early_stop.early_stop:
      print(f"Early stopping on epoch: [{epoch}/{n_epochs}]")
      break

torch.save(model, '/content/drive/MyDrive/DL_dataset/model_cats_dogs_resnet18.pth')

KeyboardInterrupt: ignored

In [ ]:
correct, total = 0, 0

model.eval()
for X_test, y_test for test_loader:
  X_test, y_test = X_test.to(device = 'cpu'), y_test.to(device = 'cpu')
  out = model(X_test)

  _, predicted = torch.argmax(out, dim = 1)
  total += y_test.shape[0]
  correct += int((predicted == y_test).sum())

print(f'Accuracy for resnet18: {correct/total')